# Save Forcing WS and precpitation for hurricane-strength events at all locations (forcing available for 2011 - 2015)

In [ ]:
# Forcing files downloaded manually from 

In [3]:
import xarray as xr
import csv
import pandas as pd
import pickle
import numpy as np
import os
import datetime #import datetime, date, timedelta
import sys
import matplotlib.pyplot as plt
import netCDF4 as nc
import os.path
import webbrowser
import time
from scipy.io import netcdf
import netCDF4
from datetime import date, timedelta, datetime


In [11]:
## Read in the data
year = 2011

filename_precip='/Users/jacoposala/Downloads/cfsv2-sea_' + str(year) + '_01hr_precip.nc' # precipitation rate
filename_windspd='/Users/jacoposala/Downloads/cfsv2-sec2_' + str(year) + '_01hr_wndspd.nc'
# filename_precip = '/Volumes/Backup/HYCOM/FORCING/cfsv2-sea_2015_01hr_precip.nc'
# filename_windspd = '/Volumes/Backup/HYCOM/FORCING/cfsv2-sec2_2015_01hr_wndspd.nc'

# HYCOM_save_dir = '/Volumes/Backup/HYCOM/DATA/HYCOM_near_TC_53x_pkl_Aug21/

ds_precip=xr.open_dataset(filename_precip,decode_times=False)
ds_windspd=xr.open_dataset(filename_windspd,decode_times=False)

varname_1 = 'wndspd' # wind
varname_1_value = 32.9244 # wind speed threshold [m/s], corresponds to 64 knots
varname_2 = 'precip' # precipitation

In [13]:
column_names = ['lat','lon','time','wind_speed','precipitation']
df_all = pd.DataFrame(columns = column_names)

start = datetime(1900,12,31,0,0,0) # This is the "days since" part

for i in ds_windspd.MT.to_numpy(): # Loop through timesteps
    print(i)
    # wind
    d_1 = ds_windspd.sel(MT=i)
    # precipitation 
    d_2 = ds_precip.sel(MT=i)
    
    # Select only time stamps and locations with WS > 64 knots
    if np.sum(d_1[varname_1] >= varname_1_value).values > 0: # ignore this time stamp if there are no hurricanes
        d_1_masked = d_1[varname_1].where(d_1[varname_1] >= varname_1_value, drop=True) 
        d_2_masked = d_2[varname_2].where(d_1[varname_1] >= varname_1_value, drop=True)

        d_1_stacked = d_1_masked.stack(x=['Latitude','Longitude'])
        d_2_stacked = d_2_masked.stack(x=['Latitude','Longitude'])

        d_1_list = d_1_stacked[d_1_stacked.notnull()]
        d_2_list = d_2_stacked[d_2_stacked.notnull()]

        # Populate DataFrame
        for j in np.arange(d_1_list.shape[0]):
            temp = pd.DataFrame(
                    {
                        # Lat
                        'lat': d_1_list.Latitude[j].values,
                        # Lon
                        'lon': d_1_list.Longitude[j].values,
                        # Time
                        'time': [start + timedelta(i)],
                        # WS
                        'wind_speed': [d_1_list[j].values],
                        # Precipitation
                        'precipitation': [d_2_list[j].values]
                    })

            df_all = pd.concat([df_all, temp])

# Show final result 
df_all

40178.0


KeyboardInterrupt: 

In [ ]:
# Save
df_all.to_csv('df_forcing_' + str(year) + '.csv')